In [1]:
!pip install torchaudio transformers librosa scikit-learn xgboost --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.2 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires rmm-cu12==24.12.*, but you have 

In [15]:
import os
import torch
import torchaudio
import librosa
import pandas as pd
import numpy as np
from torch import nn
from tqdm import tqdm
from transformers import Wav2Vec2Model, Wav2Vec2Processor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from torch.utils.data import Dataset, DataLoader
import warnings
warnings.filterwarnings("ignore")

In [3]:
train_df = pd.read_csv('/kaggle/input/shl-intern-hiring-assessment/Dataset/train.csv')
test_df = pd.read_csv('/kaggle/input/shl-intern-hiring-assessment/Dataset/test.csv')

train_path = "/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/train"
test_path = "/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/test"


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h").to(device)
model.eval()

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2Model(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): Wav2Vec2FeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Wav2Vec2Encoder(
    (pos_conv_embed): Wav2Vec2PositionalConvEmbedding(
  

In [5]:
def extract_wav2vec2_features(file_path):
    try:
        waveform, sr = torchaudio.load(file_path)
        waveform = waveform.squeeze(0)

        if sr != 16000:
            waveform = torchaudio.transforms.Resample(sr, 16000)(waveform)

        inputs = processor(waveform, sampling_rate=16000, return_tensors="pt", padding=True)
        with torch.no_grad():
            outputs = model(**inputs.to(device))
        hidden_states = outputs.last_hidden_state.squeeze(0).cpu().numpy()

        # Mean pooling across time
        feature_vector = np.mean(hidden_states, axis=0)
        return feature_vector
    except Exception as e:
        print("Error:", file_path)
        return np.zeros(768)

In [7]:
X = []
y = []
for i, row in tqdm(train_df.iterrows(), total=len(train_df)):
    file_path = os.path.join(train_path, row['filename'])
    feat = extract_wav2vec2_features(file_path)
    X.append(feat)
    y.append(row['label'])

X = np.array(X)
y = np.array(y)


 28%|██▊       | 124/444 [01:06<02:29,  2.14it/s]

Error: /kaggle/input/shl-intern-hiring-assessment/Dataset/audios/train/audio_1159.wav


 40%|███▉      | 177/444 [01:47<02:13,  1.99it/s]

Error: /kaggle/input/shl-intern-hiring-assessment/Dataset/audios/train/audio_58.wav


 65%|██████▍   | 287/444 [03:21<01:46,  1.48it/s]

Error: /kaggle/input/shl-intern-hiring-assessment/Dataset/audios/train/audio_252.wav


100%|██████████| 444/444 [06:16<00:00,  1.18it/s]


In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)

In [9]:
model_xgb = XGBRegressor(n_estimators=150, learning_rate=0.1, max_depth=6, random_state=42)
model_xgb.fit(X_train, y_train)

train_preds = model_xgb.predict(X_train)
val_preds = model_xgb.predict(X_val)

train_rmse = np.sqrt(mean_squared_error(y_train, train_preds))
val_rmse = np.sqrt(mean_squared_error(y_val, val_preds))

print(f"Train RMSE: {train_rmse:.4f}")
print(f"Validation RMSE: {val_rmse:.4f}")

Train RMSE: 0.0421
Validation RMSE: 0.8052


In [11]:
X_test = []
file_names = []

for i, row in tqdm(test_df.iterrows(), total=len(test_df)):
    file_path = os.path.join(test_path, row['filename'])
    feat = extract_wav2vec2_features(file_path)
    X_test.append(feat)
    file_names.append(row['filename'])

X_test = np.array(X_test)
test_preds = model_xgb.predict(X_test)

 55%|█████▍    | 112/204 [01:17<00:44,  2.07it/s]

Error: /kaggle/input/shl-intern-hiring-assessment/Dataset/audios/test/audio_988.wav


 62%|██████▏   | 126/204 [01:28<00:39,  1.98it/s]

Error: /kaggle/input/shl-intern-hiring-assessment/Dataset/audios/test/audio_41.wav


100%|██████████| 204/204 [02:39<00:00,  1.28it/s]


In [14]:
submission = pd.DataFrame({
    'filename': file_names,
    'label': test_preds
})
submission.to_csv('submission.csv', index=False)

**Method-2**

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
device

device(type='cuda')

In [18]:
df = pd.read_csv('/kaggle/input/shl-intern-hiring-assessment/Dataset/train.csv')
AUDIO_DIR = '/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/train'

In [19]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
wav2vec = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base").to(device)
wav2vec.eval()

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

Wav2Vec2Model(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): Wav2Vec2FeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Wav2Vec2Encoder(
    (pos_conv_embed): Wav2Vec2PositionalConvEmbedding(
  

model.safetensors:   0%|          | 0.00/380M [00:00<?, ?B/s]

In [34]:
class GrammarDataset(Dataset):
    def __init__(self, df, audio_dir, processor):
        self.df = df
        self.audio_dir = audio_dir
        self.processor = processor

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        file_path = os.path.join(self.audio_dir, row['filename'])
        try:
            y, sr = librosa.load(file_path, sr=16000)
        except:
            y = np.zeros(16000 * 5)  # silent audio fallback

        inputs = self.processor(y, sampling_rate=16000, return_tensors="pt", padding=True)
        with torch.no_grad():
            out = wav2vec(**{k: v.to(device) for k, v in inputs.items()})
            x = out.last_hidden_state.mean(dim=1).squeeze(0).cpu()  # [768-dim]
        y_score = torch.tensor(row['label'], dtype=torch.float32)
        return x, y_score

In [35]:
class GrammarRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(768, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1)
        )

    def forward(self, x):
        return self.net(x).squeeze(1)

In [36]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [37]:
train_dataset = GrammarDataset(train_df, AUDIO_DIR, processor)
val_dataset = GrammarDataset(val_df, AUDIO_DIR, processor)

In [38]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

In [39]:
model = GrammarRegressor().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()

In [40]:
EPOCHS = 20
for epoch in range(EPOCHS):
    model.train()
    train_losses = []
    for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} - Training"):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        preds = model(x)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

Epoch 20/20 - Training: 100%|██████████| 23/23 [04:54<00:00, 12.82s/it]


In [41]:
    model.eval()
    val_preds, val_labels = [], []
    with torch.no_grad():
        for x, y in tqdm(val_loader, desc="Validating"):
            x = x.to(device)
            preds = model(x)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(y.numpy())

    rmse = mean_squared_error(val_labels, val_preds, squared=False)
    print(f"Epoch {epoch+1} - Train Loss: {np.mean(train_losses):.4f} - Val RMSE: {rmse:.4f}")

Validating: 100%|██████████| 6/6 [01:15<00:00, 12.52s/it]

Epoch 20 - Train Loss: 0.7508 - Val RMSE: 0.9599


In [46]:
full_dataset = GrammarDataset(df, AUDIO_DIR, processor)
full_loader = DataLoader(full_dataset, batch_size=16)
all_preds, all_labels = [], []
model.eval()
with torch.no_grad():
    for x, y in tqdm(full_loader, desc="Evaluating Full Train Set"):
        x = x.to(device)
        preds = model(x)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(y.numpy())

rmse_final = mean_squared_error(all_labels, all_preds, squared=False)
print(f"Final RMSE on Full Train Set: {rmse_final}")

Evaluating Full Train Set: 100%|██████████| 28/28 [06:09<00:00, 13.21s/it]

Final RMSE on Full Train Set: 0.8324033617973328


In [49]:
submission = pd.DataFrame({
    "filename": test_df["filename"],
    "label": np.round(np.clip(test_preds, 0, 5)).astype(int)
})
submission.to_csv("submission_transformers_1.csv", index=False)